Reprocessing overall_scores 
Jira Ticket: https://sagebionetworks.jira.com/browse/AG-220

Tasks:
- Subtract flyneuropath scores from logsdon scores
- include ensembl_gene_id

In [11]:
import synapseclient
import pandas as pd
import numpy as np

syn = synapseclient.Synapse()
syn.login(silent=True)

# in the pipeline, we call "Logsdon" "overall"
interesting_columns = ['ENSG', 'GeneName', 'Overall', 'GeneticsScore', 'OmicsScore', 'LiteratureScore', 'FlyNeuroPathScore']

overall_scores = 'syn25575156'
overall_scores = syn.tableQuery(f'SELECT * FROM {overall_scores}').asDataFrame()
# overall_scores = overall_scores[interesting_columns]

In [12]:
# subtract the FlyNeuroPathScore from Logsdon
overall_scores['Overall'] = overall_scores['Overall'] - overall_scores['FlyNeuroPathScore']
overall_scores.drop(columns=['FlyNeuroPathScore'], inplace=True)
overall_scores.reset_index()

,index,GeneName,ENSG,Overall,Overall_rank,GeneticsScore,OmicsScore,LiteratureScore,SM_Druggability_bucket,safety_bucket,...,classification,safety_bucket_definition,feasibility_bucket_definition,ABability_bucket_definition,new_modality_bucket_definition,tissue_engagement_bucket_definition,isScored_genetics,isScored_omics,isScored_lit,isScored_neuropath
0,276953_307,GABRB3,ENSG00000166206,5.678830,1,1.897145,1.984024,1.79766153846154,1,1,...,Small molecule druggable: Protein with a small...,"Clinical data, evidence of tolerable safety pr...",High target qualification but medium feasibili...,Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is “tissue enriched, “group en...",Y,Y,Y,Y
1,276954_307,GRIN2B,ENSG00000273079,5.594506,2,2.274085,1.982329,1.33809230769231,1,1,...,Small molecule druggable: Protein with a small...,"Clinical data, evidence of tolerable safety pr...",High target qualification but medium feasibili...,Secreted protein. Highly accessible to antibod...,Not specified suitable for degradation/inhibit...,"The target gene is “tissue enriched, “group en...",Y,Y,Y,Y
2,276955_307,CDK14,ENSG00000058091,5.537711,3,1.757784,1.982266,1.79766153846154,1,3,...,Small molecule druggable: Protein with a small...,Two or fewer of: high off-target gene expressi...,"Medium target qualification, no critical safet...",Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",Y,Y,Y,Y
3,276956_307,DNAJC19,ENSG00000205981,5.536971,4,1.751060,1.988249,1.79766153846154,7,4,...,Potentially targetable by protein family struc...,More than two of: high off target gene express...,"Medium target qualification, no critical safet...",Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",Y,Y,Y,Y
4,276957_307,SCRIB,ENSG00000180900,5.418524,5,1.650295,1.970568,1.79766153846154,3,4,...,Targetable by structure: Structurally druggabl...,More than two of: high off target gene express...,High target qualification but medium feasibili...,Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24654,301607_334,NaN,ENSG00000286261,0.000000,24659,0.000000,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N
24655,301608_334,NaN,ENSG00000244313,0.000000,24659,0.000000,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N
24656,301609_334,NaN,ENSG00000271986,0.000000,24659,0.000000,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N
24657,301610_334,NaN,ENSG00000286623,0.000000,24659,0.000000,0.000000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N


In [13]:
print(overall_scores.columns)
scored = ['isScored_genetics', 'isScored_omics', 'isScored_lit']
mapping = dict(zip(interesting_columns[3:], scored))

# for metric in scored:
#     overall_scores[metric] = np.nan
    
# overall_scores.reset_index()


Index(['GeneName', 'ENSG', 'Overall', 'Overall_rank', 'GeneticsScore',
       'OmicsScore', 'LiteratureScore', 'SM_Druggability_bucket',
       'safety_bucket', 'feasibility_bucket', 'ABability_bucket',
       'new_modality_bucket', 'tissue_engagement_bucket', 'Pharos_class',
       'classification', 'safety_bucket_definition',
       'feasibility_bucket_definition', 'ABability_bucket_definition',
       'new_modality_bucket_definition', 'tissue_engagement_bucket_definition',
       'isScored_genetics', 'isScored_omics', 'isScored_lit',
       'isScored_neuropath'],
      dtype='object')


In [14]:
for field, is_scored in mapping.items():
    overall_scores.loc[lambda row: row[is_scored] == 'N', field] = np.nan

overall_scores.reset_index()

,index,GeneName,ENSG,Overall,Overall_rank,GeneticsScore,OmicsScore,LiteratureScore,SM_Druggability_bucket,safety_bucket,...,classification,safety_bucket_definition,feasibility_bucket_definition,ABability_bucket_definition,new_modality_bucket_definition,tissue_engagement_bucket_definition,isScored_genetics,isScored_omics,isScored_lit,isScored_neuropath
0,276953_307,GABRB3,ENSG00000166206,5.678830,1,1.897145,1.984024,1.79766153846154,1,1,...,Small molecule druggable: Protein with a small...,"Clinical data, evidence of tolerable safety pr...",High target qualification but medium feasibili...,Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is “tissue enriched, “group en...",Y,Y,Y,Y
1,276954_307,GRIN2B,ENSG00000273079,5.594506,2,2.274085,1.982329,1.33809230769231,1,1,...,Small molecule druggable: Protein with a small...,"Clinical data, evidence of tolerable safety pr...",High target qualification but medium feasibili...,Secreted protein. Highly accessible to antibod...,Not specified suitable for degradation/inhibit...,"The target gene is “tissue enriched, “group en...",Y,Y,Y,Y
2,276955_307,CDK14,ENSG00000058091,5.537711,3,1.757784,1.982266,1.79766153846154,1,3,...,Small molecule druggable: Protein with a small...,Two or fewer of: high off-target gene expressi...,"Medium target qualification, no critical safet...",Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",Y,Y,Y,Y
3,276956_307,DNAJC19,ENSG00000205981,5.536971,4,1.751060,1.988249,1.79766153846154,7,4,...,Potentially targetable by protein family struc...,More than two of: high off target gene express...,"Medium target qualification, no critical safet...",Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",Y,Y,Y,Y
4,276957_307,SCRIB,ENSG00000180900,5.418524,5,1.650295,1.970568,1.79766153846154,3,4,...,Targetable by structure: Structurally druggabl...,More than two of: high off target gene express...,High target qualification but medium feasibili...,Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24654,301607_334,NaN,ENSG00000286261,0.000000,24659,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N
24655,301608_334,NaN,ENSG00000244313,0.000000,24659,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N
24656,301609_334,NaN,ENSG00000271986,0.000000,24659,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N
24657,301610_334,NaN,ENSG00000286623,0.000000,24659,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N,N


In [18]:
overall_scores['LiteratureScore'] = pd.to_numeric(overall_scores['LiteratureScore'])


In [ ]:
overall_scores.to_json('../output/overall_scores.json', orient='records', indent=2)